In [20]:
import numpy as np
import cv2
import math

WIND_X = 600
WIND_Y = 600
cv2.namedWindow  ("tezd", cv2.WINDOW_AUTOSIZE)
cv2.resizeWindow ("tezd", (WIND_Y, WIND_X))
canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 200

x0 = 180
y0 = 120
points_num = 10
r = 100

#field_points = {200: {20: (10, 10), 340: (50, 10)},
#                300: {20: (30, 35), 50: (40, 60)},
#                400: {20: (330, 135), 50: (240, 260)},
#                500: {70: (390, 135), 120: (240, 260)}}

def twopixonefield (x0, y0):
    field_points = {257: {120: (27, -10)},
                    253: {123: (27, 10)},
                    264: {120: (26, 29)},

                    276: {196: (37, -19)},
                    176: {198: (37, 0)},
                    95: {197: (37, 15)},
                    42: {192: (37, 28)},

                    267: {146: (50, -25)},
                    213: {144: (51, -5)},
                    107: {142: (52, 15)},
                    31: {140: (52, 35)},

                    301: {104: (66, -31)},
                    249: {103: (67, -19)},
                    186: {102: (67, -4)},
                    139: {102: (67, 9)},
                    87: {100: (67, 25)},
                    36: {102: (67, 45)},

                    291: {68: (90, -44)},
                    260: {66: (90, -30)},
                    211: {64: (90, -15)},
                    163: {64: (90, 0)},
                    137: {64: (90, 12)},
                    102: {63: (90, 29)},
                    55: {64: (90, 43)},
                    12: {66: (90, 60)},

                    306: {48: (111, -55)},
                    257: {45: (110, -32)},
                    215: {46: (110, -12)},
                    168: {44: (110, 4)},
                    120: {44: (110, 20)},
                    73: {44: (110, 40)},
                    23: {48: (110, 67)},

                    307: {35: (129, -65)},
                    273: {32: (129, -47)},
                    238: {28: (129, -29)},
                    202: {30: (129, -13)},
                    167: {28: (129, 3)},
                    125: {30: (128, 21)},
                    86: {31: (130, 42)},
                    45: {35: (129, 64)},
                    12: {38: (129, 85)},

                    299: {23: (150, -70)},
                    265: {20: (150, -49)},
                    239: {19: (150, -35)},
                    210: {17: (150, -19)},
                    181: {17: (150, -5)},
                    158: {18: (150, 9)},
                    131: {18: (150, 22)},
                    101: {19: (150, 39)},
                    73: {21: (150, 55)},
                    52: {23: (150, 69)},
                    32: {25: (150, 81)},
                    11: {27: (150, 99)}}

    print (len (field_points))

    min_dist = 100500
    min_row = 0
    min_point = 0

    for row in field_points.keys ():
        for point in field_points [row]:
            p = field_points [row] [point]

            cv2.circle (canvas, (int (row), int (point)), 5, (130, 170, 130), -1)

            dist = (int (row) - x0)**2 + (int (point) - y0)**2

            if (dist < min_dist):
                print ("new min found")
                min_dist = dist
                min_row = row
                min_point = point

    cv2.circle (canvas, (x0, y0), 7, (230, 7, 230), -1)
    cv2.circle (canvas, (int (min_row), int (min_point)), 7, (230, 170, 230), -1)

twopixonefield (x0, y0)

cv2.imshow ("tezd", canvas)
cv2.waitKey           (1)
cv2.destroyAllWindows ()

50
new min found
new min found
new min found
new min found


# Домашнее задание 1

In [1]:
import cv2
import time
import os
import math
import sys

sys.path.append("../modules/")

import input_output
import processor

det = processor.Processors ()
det.add_processor ("keypoints extr")

det.add_filter (processor.resize (downscale_factor_ = 1),
                
                "keypoints extr", "downscale")
det.add_filter (processor.colorspace_to_colorspace ("RGB", "GRAY"),
                "keypoints extr", "colorspaces change")

bsz = 5 # block size
asz = 3 # aperture size
k   = 0.04 # alg parameter

def custom_harris (img):
    dst = cv2.cornerHarris (img, bsz, asz, k)
    img[dst > 0.01 * dst.max ()] = 255
    return img

det.add_filter (processor.custom_operation (custom_harris, "harris filter"), "keypoints extr", "harris filter")

source  = input_output.Source ("-1")

while (True):
    frame = source.get_frame ()

    result, success = det.process (frame, "keypoints extr")

    cv2.imshow ("harris", result)
    
    time.sleep (0.02)

    keyb = cv2.waitKey (1) & 0xFF

    if (keyb == ord('q')):
        break

source.release ()
cv2.waitKey (1)
cv2.destroyAllWindows ()

In [3]:
import cv2
import time
import os
import math
import sys

sys.path.append("../modules/")

import input_output
import processor

proc_name = "palm detection"

det = processor.Processors()
det.add_processor(proc_name)

#gamma correction (lighting conditions change simulation) and online tuning
det.add_filter(processor.gamma_correction(1.2), proc_name, "gamma correction")

window_name = 'trackbars'
cv2.namedWindow(window_name, cv2.WINDOW_AUTOSIZE)

# HSV colorspace
det.add_filter (processor.colorspace_to_colorspace("RGB", "HSV"),
                proc_name, "colorspaces change")

# inrange for HSV with trackbars for tuning
det.add_filter(processor.inrange((115, 5, 170), (185, 70, 240)), proc_name, "inrange hsv")

def change_thr(new_val, which, val_nb):
    low_th = list(det.processors[proc_name]["inrange hsv"].low_th)
    high_th = list(det.processors[proc_name]["inrange hsv"].high_th)
    if which == 'low':
        low_th[val_nb] = new_val
    else:
        high_th[val_nb] = new_val
    det.processors[proc_name]["inrange hsv"].set_ths(tuple(low_th), tuple(high_th))

cv2.createTrackbar('lowH', window_name, 115, 255,lambda x : change_thr(x, 'low', 0)) # 0 for S
cv2.createTrackbar('highH', window_name, 185, 255, lambda x : change_thr(x, 'high', 0))
cv2.createTrackbar('lowS', window_name, 5, 255, lambda x : change_thr(x, 'low', 1)) # 1 for H
cv2.createTrackbar('highS', window_name, 70, 255, lambda x : change_thr(x, 'high', 1))
cv2.createTrackbar('lowV', window_name, 170, 255, lambda x : change_thr(x, 'low', 2)) # 2 for V
cv2.createTrackbar('highV', window_name, 240, 255, lambda x : change_thr(x, 'high', 2))

det.add_filter(processor.morphology("close", 11), proc_name, "close")

det.add_filter(processor.filter_connected_components(area_low_ = 1000,
               den_high_ = 0.55), proc_name, "filter cc")

#bbox extraction
det.add_filter(processor.max_area_cc_bbox(), proc_name, "bbox extraction")

source  = input_output.Source("-1")

#create_video_writer = True
#FPS = 10

while True:
    frame = source.get_frame ()

    det.process (frame, proc_name)
    stages = det.get_stages_picts (proc_name)
    
    cv2.imshow ("stages", input_output.form_grid (stages, window_x_sz=800))
    
    # saving the video
    #if create_video_writer:
        #height, width, _ = result.shape
        #out = cv2.VideoWriter('hand detected.avi',
        #      cv2.VideoWriter_fourcc('M','J','P','G'), FPS, (width, height))
        #create_video_writer = False
    
    #out.write(result)
    
    time.sleep(0.002)

    keyb = cv2.waitKey(1) & 0xFF

    if keyb == ord('q'):
        break

#out.release()
cv2.destroyAllWindows()